In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os, json
from typing import Annotated, TypedDict
import operator
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage, AnyMessage
from langchain_community.tools import TavilySearchResults
from langgraph.graph import StateGraph, END

In [4]:
tool = TavilySearchResults(tavily_api_key=os.getenv("TAVILY_API_KEY"),max_results=2)

In [5]:
print(tool)

max_results=2 api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))


In [6]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage],operator.add]

In [7]:
from langgraph.checkpoint.sqlite import SqliteSaver

In [8]:
memory = SqliteSaver.from_conn_string(":memory:")

In [9]:
class Agent:
    def __init__(self,model, tools, checkpointer, system = ""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm",self.call_openai)
        graph.add_node("action",self.take_action)
        graph.add_conditional_edges("llm", self.exist_action, {True: "action", False: END})
        graph.add_edge("action","llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer = checkpointer)
        self.model = model.bind_tools(tools)
        self.tools = {t.name: t for t in tools}

    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages":[message]}
    
    def exist_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0
    
    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling tool {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(ToolMessage(tool_call_id=t["id"],name=t["name"],content=str(result)))
        print("Back to model")
        return {"messages":results}

In [10]:
import contextlib

In [11]:
checkpointer_cm = SqliteSaver.from_conn_string(":memory:")
checkpointer = contextlib.ExitStack().enter_context(checkpointer_cm)

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o",api_key=os.getenv("OPENAI_API_KEY"))
abot = Agent(model, [tool], checkpointer=checkpointer, system=prompt)

In [13]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [14]:
thread = {"configurable": {"thread_id": "1"}}

In [15]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NvLz1Jex1sebnWbvy013aYQU', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 151, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_92f14e8683', 'id': 'chatcmpl-BLDG1EUpm7KRqDWdoSImm4DnufX24', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7d918455-86ce-4535-9af0-cb2e5603ee34-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_NvLz1Jex1sebnWbvy013aYQU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens'

In [26]:
# with SqliteSaver.from_conn_string(":memory:") as checkpointer:
#     abot = Agent(model, [tool], checkpointer=checkpointer, system=prompt)

#     messages = [HumanMessage(content="What is the weather in sf?")]
#     thread = {"configurable": {"thread_id": "1"}}

#     for event in abot.graph.stream({"messages": messages}, thread):
#         for v in event.values():
#             print(v["messages"])

In [27]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="To determine which temperature is warmer, let's compare:\n\n- San Francisco's current temperature: 11.1°C (52.0°F).\n\nSince there is no other location mentioned for comparison, it's not possible to determine which one is warmer without another reference point. If you have another location or temperature in mind for comparison, please let me know!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 795, 'total_tokens': 866, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BKoG9mdWfg1cM6lmEIfahPPnOh8YQ', 'finish_reason': 'stop', 'logprobs': None}, id='run-062703f6-5c45-45f6-86d4-90818063d4a2-0', usage_metadata={'input_tokens': 795, 'output_tokens': 

In [28]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please clarify what you are comparing to determine which is warmer? Are you asking about specific locations, time periods, clothing materials, or something else?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 149, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BKoHvjrmA4xAPppIqXY2tSirlkHby', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc164f65-692b-418c-9254-8acaca466530-0', usage_metadata={'input_tokens': 149, 'output_tokens': 33, 'total_tokens': 182, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [30]:
import contextlib
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

stack = contextlib.AsyncExitStack()
checkpointer = await stack.enter_async_context(AsyncSqliteSaver.from_conn_string(":memory:"))


In [31]:
abot = Agent(model, [tool], system=prompt, checkpointer=checkpointer)

In [32]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling tool {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_DIScswWAI1xWyEYVjCiUY59W', 'type': 'tool_call'}
Back to model
